In [1]:
# If you have installation questions, please reach out

import seaborn as sns
import pandas as pd # data storage
import catboost as cats # graident boosting 

from catboost import CatBoostRegressor, Pool

import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

print('catboost ver:', cats.__version__)
print('scikit ver:', sklearn.__version__)

import datetime

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import median_absolute_error, max_error, mean_squared_error

catboost ver: 0.26.1
scikit ver: 0.24.2


In [2]:
from catboostmodels import catboost_multi_xrf

# dataframes

In [3]:
df0 = pd.read_csv('../../core_to_wl_merge/OS0_Merged_dataset_imputed_08_23_2021.csv')

df1 = pd.read_csv('../../core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

df2 = pd.read_csv('../../core_to_wl_merge/OS2_Merged_dataset_imputed_08_23_2021.csv')

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
param_dict ={
    "iter" : 1000,       #Maximum iterations
    "GPUorCPU": "CPU", #CPU is fine for this dataset on most laptops
    "inputs": ['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE', 'RESD', 'PHIN', 'PHID', 'GR_smooth',  'PE_smooth'],
    "target": ['Ti', 'Mg', 'Al', 'Ca', 'Si' ]}

# Offset 0

In [ ]:
dataset = df[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'Ti', 'Mg', 'Al', 'Ca', 'Si' ]]

In [ ]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
dataset = dataset.dropna()

# dataset.head(3)

In [ ]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

Y2 = dataset[['Ti', 'Mg', 'Al', 'Ca', 'Si']]

In [ ]:
seed = 42 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y2.values, test_size=test_size)

# iterations

In [ ]:
max_iter=500

In [ ]:
df_OS0 = catboost_multi_xrf(X_train, X_test, y_train, y_test, 'catboost_MultiXRF.csv', max_iter)

# Offset 1

In [ ]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df2 = df2[df2.Al >= 0]

dataset2 = df2[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'Ti', 'Mg', 'Al', 'Ca', 'Si']]

# Features we will use for prediction
X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y2 = dataset2[['Ti', 'Mg', 'Al', 'Ca', 'Si']]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2.values, Y2.values, test_size=test_size)

In [ ]:
catboost_multi_xrf(X_train2, X_test2, y_train2, y_test2, 'OS1_catboost_MultiXRF.csv', max_iter)

# Offset 2

In [ ]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df3 = df3[df3.Al >= 0]

dataset3 = df3[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'Ti', 'Mg', 'Al', 'Ca', 'Si']]

# Features we will use for prediction
X3 = dataset3[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y3 = dataset2[['Ti', 'Mg', 'Al', 'Ca', 'Si']]

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3.values, Y3.values, test_size=test_size)

In [ ]:
catboost_multi_xrf(X_train3, X_test3, y_train3, y_test3, 'OS2_catboost_MultiXRF.csv', max_iter)